# CNN Medal

The aim of this exercise is to use the previous dataset explored in CNN.ipynb file and improve it.

rules :
* can't modify the test and training size
* Dropout admitted
* Changing optimizer admitted
* Don't specify a specific seed

Medal :
* Bronze : between 80 to 85 % on test dataset
* Silver : 85 to 90%
* Gold : Upper 90%


In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# scikit
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
# keras
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Convolution2D, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

Using TensorFlow backend.


In [58]:
class Model_CNN():
    def __init__(self, cnn_layers, dense_layers, optimizer, loss, metrics, 
             input_shape, load = False, name = "model"):
        if load == False:
            self._epochDone = 0
            self._currentAccuracy = 0
            self._batch_size = 32
            self._training_size = 8000
            self._test_size = 2000
            self._image_size = (input_shape[0],input_shape[1])
            self._model = Sequential()
            # cnn layers
            i = 0
            for layer in cnn_layers:
                # input layer or not
                if i == 0:
                    self._model.add(Convolution2D(filters=layer["filters"], 
                        kernel_size=layer["kernel_size"], strides=layer["strides"], 
                        input_shape=input_shape, activation=layer["activation"]))
                else:
                    self._model.add(Convolution2D(filters=layer["filters"], 
                        kernel_size=layer["kernel_size"], strides=layer["strides"], 
                        activation=layer["activation"]))
                # max pooling
                if layer["maxpooling"][0]:
                    self._model.add(MaxPooling2D(pool_size=layer["maxpooling"][1]))
                # dropout
                if layer["dropout"][0]:
                    self._model.add(Dropout(layer["dropout"][1]))
                i+=1
            self._model.add(Flatten())

            # dense layers
            for layer in dense_layers:
                self._model.add(Dense(units=layer["units"], activation=layer["activation"]))
                # dropout
                if layer["dropout"][0]:
                    self._model.add(Dropout(layer["dropout"][1]))

            self._model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        else:
            self.load_network(name)
        self._model.summary()
            
        
    def train(self, epochs, name):
        train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2,
                                zoom_range=0.2,horizontal_flip=True)

        # same for the test set
        test_datagen = ImageDataGenerator(rescale=1./255)

        # generate the new images for train dataset
        train_generator = train_datagen.flow_from_directory(
                    'training_set', target_size=self._image_size,
                    batch_size=self._batch_size, class_mode='binary')
        # same for the test dataset
        test_generator = test_datagen.flow_from_directory(
                    'test_set', target_size=self._image_size,
                    batch_size=self._batch_size, class_mode='binary')
        for i in range(epochs):
            # do the job
            values = self._model.fit(train_generator,
                    steps_per_epoch=int(self._training_size/self._batch_size),
                    epochs=1,
                    validation_data=test_generator,
                    validation_steps=int(self._test_size/self._batch_size))
            self._currentAccuracy = values.history['val_accuracy']
            self._epochDone += 1
            self.save_network(name)
            
    
    def save_network(self, name):
        try:
            self._model.save('model\\{}'.format(name))
            network = {
                "_epochDone":self._epochDone,
                "_currentAccuracy":self._currentAccuracy,
                "_batch_size":self._batch_size,
                "_training_size":self._training_size,
                "_test_size":self._test_size,
                "_image_size":self._image_size,
                "_name":name,
            }
            with open('model\\{}_json.json'.format(name), 'w') as outfile:
                json.dump(network, outfile)
            print("Saving Done Correctly")
        except Exception as inst:
            print("Saving Failed")
            print(inst)
        
            
    def load_network(self, name):
        try:
            self._model = keras.models.load_model('model\\{}'.format(name))
            with open('model\\{}_json.json'.format(name)) as json_file:
                data = json.load(json_file)
                self._epochDone = data["_epochDone"]
                self._currentAccuracy = data["_currentAccuracy"]
                self._batch_size = data["_batch_size"]
                self._training_size = data["_training_size"]
                self._test_size = data["_test_size"]
                self._image_size = data["_image_size"]
                self._name = data["_name"]
            print("Loading Done Correctly")
        except Exception as inst:
            print("Loading Failed")
            print(inst)
    
    def current_accuracy(self):
        return self._currentAccuracy
        

In [61]:
def init_1():
    layers_cnn = [
        {
            "filters":32,"kernel_size":(3,3),"strides":(1,1),
            "activation":"relu","maxpooling":[True,(2,2)],"dropout":[False]
        },
        {
            "filters":32,"kernel_size":(3,3),"strides":(1,1),
            "activation":"relu","maxpooling":[True,(2,2)],"dropout":[False]
        },
        {
            "filters":64,"kernel_size":(3,3),"strides":(1,1),
            "activation":"relu","maxpooling":[True,(2,2)],"dropout":[False]
        },
        {
            "filters":64,"kernel_size":(3,3),"strides":(1,1),
            "activation":"relu","maxpooling":[True,(2,2)],"dropout":[False]
        }
    ]

    layers_dense = [
        {
            "units":128,
            "activation":"relu","dropout":[False]
        },
        {
            "units":64,
            "activation":"relu","dropout":[False]
        },
        {
            "units":32,
            "activation":"relu","dropout":[True, 0.3]
        },
        {
            "units":1,
            "activation":"sigmoid","dropout":[False]
        },
    ]
    model_1 = Model_CNN(layers_cnn, layers_dense, "adam", 
                    "binary_crossentropy", ["accuracy"], (150,150,3))
    return model_1

### Loading Model process

Use the following lines to load your model

In [59]:
# creating for the first time your model
# model_1 = init_1()
# loading it from existing file in the model folder
model_1 = Model_CNN(None, None, None, None, None, None, True, "model1")

Loading Done Correctly
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (N

### Training process

Execute all the following lines to train your model.

It will be saved every epoch.

In [45]:
epoch_number = 1

In [60]:
print("The model has been already trained {} epochs".format(model_1._epochDone))
print("The current accuracy over the test set is {}".format(model_1.current_accuracy()))

The model has been already trained 1 epochs
The current accuracy over the test set is [0.5675403475761414]


In [56]:
model_1.train(epoch_number, "model1")

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/1
250/250 [==============================] - 112s 447ms/step - loss: 0.6914 - accuracy: 0.5231 - val_loss: 0.6716 - val_accuracy: 0.5675
Saving Done Correctly
